In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

df = pd.read_csv('Datasets/cleaned_data.csv')

# Round the mood column to nearest integer
df['mood'] = df['mood'].round()
df['date'] = pd.to_datetime(df['date'])

# Extract year, month, day, hour, and minute as separate columns
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day

# Perform one-hot encoding one the persons
one_hot_df = pd.get_dummies(df['id'])

# Concatenate the one-hot encoded columns to the original DataFrame
df = pd.concat([df, one_hot_df], axis=1)

# Drop the original categorical column
df.drop(['id'], axis=1, inplace=True)

In [27]:
# Define the input and target variables
X = df.drop(["date", "mood"], axis=1) # drop the date and target columns
y = df["mood"]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features using StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Encode target variable using LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Reshape input data to fit RNN input shape
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [30]:
# Define hyperparameters
lstm_size = 100
num_layers = 2
dropout = 0.2
optimizer = 'adam'
eps = 50
batch = 64

# Define RNN model architecture
model = Sequential()    
for i in range(num_layers):
    model.add(LSTM(units=lstm_size, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout))
    
model.add(LSTM(units=lstm_size))
model.add(Dropout(dropout))
model.add(Dense(units=10, activation='softmax'))

# Compile model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=eps, batch_size=batch, validation_data=(X_test, y_test))

# Evaluate model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


Epoch 1/50
17/17 [==============================] - 7s 94ms/step - loss: 2.2726 - accuracy: 0.4722 - val_loss: 2.2234 - val_accuracy: 0.5594
Epoch 2/50
17/17 [==============================] - 0s 28ms/step - loss: 2.1512 - accuracy: 0.5307 - val_loss: 1.9889 - val_accuracy: 0.5632
Epoch 3/50
17/17 [==============================] - 0s 29ms/step - loss: 1.7416 - accuracy: 0.5287 - val_loss: 1.3219 - val_accuracy: 0.5517
Epoch 4/50
17/17 [==============================] - 0s 28ms/step - loss: 1.2233 - accuracy: 0.4981 - val_loss: 1.0811 - val_accuracy: 0.5632
Epoch 5/50
17/17 [==============================] - 0s 28ms/step - loss: 1.0674 - accuracy: 0.5508 - val_loss: 1.0333 - val_accuracy: 0.5441
Epoch 6/50
17/17 [==============================] - 0s 28ms/step - loss: 0.9997 - accuracy: 0.5785 - val_loss: 0.9780 - val_accuracy: 0.5632
Epoch 7/50
17/17 [==============================] - 0s 28ms/step - loss: 0.9211 - accuracy: 0.6169 - val_loss: 0.9210 - val_accuracy: 0.5977
Epoch 8/50
17

In [31]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


# Define function to create RNN model
def create_model(lstm_size=50, num_layers=2, dropout=0.5, optimizer='adam'):
    model = Sequential()
    for i in range(num_layers):
        model.add(LSTM(units=lstm_size, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(dropout))
    model.add(LSTM(units=lstm_size))
    model.add(Dropout(dropout))
    model.add(Dense(units=10, activation='softmax'))
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create KerasClassifier object
model = KerasClassifier(build_fn=create_model)

# Create dictionary for hyperparameters
param_grid = {
    'lstm_size': [25, 50, 100],
    'num_layers': [2, 3, 4],
    'dropout': [0.2, 0.3, 0.4, 0.5],
    'batch_size': [16, 32, 64],
    'epochs': [50, 100, 150],
    'optimizer': ['adam']
}

# Create GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Fit GridSearchCV object to data
grid_result = grid.fit(X_train, y_train)

# Print results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


C:\Users\noahv\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Epoch 1/50
44/44 [==============================] - 5s 6ms/step - loss: 2.2589 - accuracy: 0.4713
Epoch 2/50
44/44 [==============================] - 0s 5ms/step - loss: 2.1093 - accuracy: 0.5172
Epoch 3/50
44/44 [==============================] - 0s 5ms/step - loss: 1.6123 - accuracy: 0.5172
Epoch 4/50
44/44 [==============================] - 0s 5ms/step - loss: 1.2075 - accuracy: 0.5172
Epoch 5/50
44/44 [==============================] - 0s 5ms/step - loss: 1.1424 - accuracy: 0.5172
Epoch 6/50
44/44 [==============================] - 0s 6ms/step - loss: 1.0988 - accuracy: 0.5172
Epoch 7/50
44/44 [==============================] - 0s 5ms/step - loss: 1.0791 - accuracy: 0.5172
Epoch 8/50
44/44 [==============================] - 0s 7ms/step - loss: 1.0499 - accuracy: 0.5330
Epoch 9/50
44/44 [==============================] - 0s 5ms/step - loss: 1.0058 - accuracy: 0.5517
Epoch 10/50
44/44 [==============================] - 0s 5ms/step - loss: 0.9688 - accuracy: 0.5819
Epoch 11/50
44/44 [

KeyboardInterrupt: 

In [28]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


# Define function to create RNN model
def create_model(lstm_size=50, num_layers=2, dropout=0.5, optimizer='adam'):
    model = Sequential()
    for i in range(num_layers):
        model.add(LSTM(units=lstm_size, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(dropout))
    model.add(LSTM(units=lstm_size))
    model.add(Dropout(dropout))
    model.add(Dense(units=10, activation='softmax'))
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create KerasClassifier object
model = KerasClassifier(build_fn=create_model)

# Define hyperparameters to tune
lstm_sizes = [25, 50, 100]
num_layers = [2, 3, 4]
dropouts = [0.2, 0.3, 0.4, 0.5]
optimizers = ['adam']

# Define hyperparameter grid
param_grid = dict(lstm_size=lstm_sizes, num_layers=num_layers, dropout=dropouts, optimizer=optimizers)

# Create GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Fit GridSearchCV object to data
grid_result = grid.fit(X_train, y_train)

# Print results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


C:\Users\noahv\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


33/33 [==============================] - 8s 20ms/step - loss: 2.2414 - accuracy: 0.4761
Best: 0.523946 using {'dropout': 0.2, 'lstm_size': 100, 'num_layers': 2, 'optimizer': 'adam'}


In [23]:
# Define the input and target variables
X = df.drop(["date", "mood"], axis=1) # drop the date and target columns
y = df["mood"]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features using StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Encode target variable using LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Reshape input data to fit RNN input shape
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Define RNN model architecture
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(units=50))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test))

# Evaluate model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


Epoch 1/200
17/17 [==============================] - 28s 187ms/step - loss: 2.2806 - accuracy: 0.4732 - val_loss: 2.2511 - val_accuracy: 0.5479
Epoch 2/200
17/17 [==============================] - 0s 21ms/step - loss: 2.2331 - accuracy: 0.4933 - val_loss: 2.2020 - val_accuracy: 0.5479
Epoch 3/200
17/17 [==============================] - 0s 19ms/step - loss: 2.1880 - accuracy: 0.4933 - val_loss: 2.1542 - val_accuracy: 0.5479
Epoch 4/200
17/17 [==============================] - 0s 23ms/step - loss: 2.1441 - accuracy: 0.4933 - val_loss: 2.1085 - val_accuracy: 0.5479
Epoch 5/200
17/17 [==============================] - 0s 26ms/step - loss: 2.1023 - accuracy: 0.4933 - val_loss: 2.0647 - val_accuracy: 0.5479
Epoch 6/200
17/17 [==============================] - 0s 22ms/step - loss: 2.0620 - accuracy: 0.4933 - val_loss: 2.0228 - val_accuracy: 0.5479
Epoch 7/200
17/17 [==============================] - 0s 22ms/step - loss: 2.0246 - accuracy: 0.4933 - val_loss: 1.9830 - val_accuracy: 0.5479
Epoc